In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the earthquake data
earthquake_data = pd.read_csv("C:/Users/galia/Downloads/archive (11)/india.csv")

# Remove timezone information from 'Origin Time' and convert to datetime
earthquake_data['Origin Time'] = pd.to_datetime(earthquake_data['Origin Time'].str.replace(r'\s\w+$', '', regex=True))

# Extract numerical features from the datetime objects
earthquake_data['Year'] = earthquake_data['Origin Time'].dt.year
earthquake_data['Month'] = earthquake_data['Origin Time'].dt.month
earthquake_data['Day'] = earthquake_data['Origin Time'].dt.day
earthquake_data['Hour'] = earthquake_data['Origin Time'].dt.hour
earthquake_data['Minute'] = earthquake_data['Origin Time'].dt.minute
earthquake_data['Second'] = earthquake_data['Origin Time'].dt.second

# Extract relevant features and targets for prediction
X = earthquake_data[['Latitude', 'Longitude', 'Depth', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']]
y_magnitude = earthquake_data['Magnitude']
y_time = earthquake_data['Origin Time']

# Scale the features and targets
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
scaler_y_magnitude = MinMaxScaler()
y_magnitude_scaled = scaler_y_magnitude.fit_transform(np.array(y_magnitude).reshape(-1, 1))
scaler_y_time = MinMaxScaler()
y_time_scaled = scaler_y_time.fit_transform(y_time.astype('int64').values.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train_magnitude, y_test_magnitude, y_train_time, y_test_time = train_test_split(
    X_scaled, y_magnitude_scaled, y_time_scaled, test_size=0.2, random_state=42
)

# Reshape input data for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

def build_and_train_model(X_train, y_train, X_test, y_test, target_name):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
    train_loss = model.evaluate(X_train, y_train)
    test_loss = model.evaluate(X_test, y_test)
    predicted = model.predict(X_test)
    return model, train_loss, test_loss, predicted

# Build, train, and evaluate magnitude model
model_magnitude, train_loss_magnitude, test_loss_magnitude, predicted_magnitude = build_and_train_model(
    X_train, y_train_magnitude, X_test, y_test_magnitude, 'magnitude'
)

# Inverse transform magnitude predictions to get original scale
predicted_magnitude_original_scale = scaler_y_magnitude.inverse_transform(predicted_magnitude)
print("Next predicted earthquake magnitude:", predicted_magnitude_original_scale[-1][0])

# Build, train, and evaluate time model
model_time, train_loss_time, test_loss_time, predicted_time = build_and_train_model(
    X_train, y_train_time, X_test, y_test_time, 'time'
)

# Inverse transform time predictions to get original scale
predicted_time_original_scale = scaler_y_time.inverse_transform(predicted_time)
predicted_time_timestamp = int(predicted_time_original_scale[-1][0])

# Convert timestamp to datetime
predicted_time_datetime = pd.to_datetime(predicted_time_timestamp, unit='ns')
print("Next predicted earthquake time:", predicted_time_datetime)


C:\Users\galia\.anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.1107 - val_loss: 0.0201
Epoch 2/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0206 - val_loss: 0.0198
Epoch 3/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0188 - val_loss: 0.0193
Epoch 4/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0193 - val_loss: 0.0193
Epoch 5/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0204 - val_loss: 0.0189
Epoch 6/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0180 - val_loss: 0.0183
Epoch 7/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0184 - val_loss: 0.0179
Epoch 8/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0172 - val_loss: 0.0170
Epoch 9/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0165 - val_loss: 0.0167
Epoch 10/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0166 - val_loss: 0.0171
Epoch 11/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0165 - val_loss: 0.0179
Epoch 12/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0159 - val_l

C:\Users\galia\.anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.2031 - val_loss: 0.0614
Epoch 2/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0593 - val_loss: 0.0589
Epoch 3/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0560 - val_loss: 0.0550
Epoch 4/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0514 - val_loss: 0.0511
Epoch 5/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0466 - val_loss: 0.0444
Epoch 6/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0419 - val_loss: 0.0450
Epoch 7/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0427 - val_loss: 0.0412
Epoch 8/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0364 - val_loss: 0.0390
Epoch 9/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0379 - val_loss: 0.0362
Epoch 10/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0319 - val_loss: 0.0316
Epoch 11/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0253 - val_loss: 0.0264
Epoch 12/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0242 - val_loss: 0.0231
